<a href="https://colab.research.google.com/github/srijhanavi/FMML_LABS_AND_PROJECTS/blob/main/Module_01_Lab_02_MLPractice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine learning terms and metrics

FMML Module 1, Lab 2<br>


 In this lab, we will show a part of the ML pipeline by extracting features, training and testing

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from sklearn import datasets
# set randomseed
rng = np.random.default_rng(seed=42)

In this lab, we will use the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district.

Let us download and examine the dataset.

In [2]:
 dataset =  datasets.fetch_california_housing()
 # print(dataset.DESCR)  # uncomment this if you want to know more about this dataset
 # print(dataset.keys())  # if you want to know what else is there in this dataset
 dataset.target = dataset.target.astype(np.int) # so that we can classify
 print(dataset.data.shape)
 print(dataset.target.shape)

(20640, 8)
(20640,)


<ipython-input-2-60ae2e9a125e>:4: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dataset.target = dataset.target.astype(np.int) # so that we can classify


Here is a function for calculating the 1-nearest neighbours

In [3]:
def NN1(traindata, trainlabel, query):
  diff  = traindata - query  # find the difference between features. Numpy automatically takes care of the size here
  sq = diff*diff # square the differences
  dist = sq.sum(1) # add up the squares
  label = trainlabel[np.argmin(dist)] # our predicted label is the label of the training data which has the least distance from the query
  return label

def NN(traindata, trainlabel, testdata):
  # we will run nearest neighbour for each sample in the test data
  # and collect the predicted classes in an array using list comprehension
  predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
  return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [4]:
def RandomClassifier(traindata, trainlabel, testdata):
  # in reality, we don't need these arguments

  classes = np.unique(trainlabel)
  rints = rng.integers(low=0, high=len(classes), size=len(testdata))
  predlabel = classes[rints]
  return predlabel

Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm.

In [5]:
def Accuracy(gtlabel, predlabel):
  assert len(gtlabel)==len(predlabel), "Length of the groundtruth labels and predicted labels should be the same"
  correct = (gtlabel==predlabel).sum() # count the number of times the groundtruth label is equal to the predicted label.
  return correct/len(gtlabel)

Let us make a function to split the dataset with the desired probability.

In [6]:
def split(data, label, percent):
  # generate a random number for each sample
  rnd = rng.random(len(label))
  split1 = rnd<percent
  split2 = rnd>=percent
  split1data = data[split1,:]
  split1label = label[split1]
  split2data = data[split2,:]
  split2label = label[split2]
  return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [7]:
testdata, testlabel, alltraindata, alltrainlabel = split(dataset.data, dataset.target, 20/100)
print('Number of test samples = ', len(testlabel))
print('Number of other samples = ', len(alltrainlabel))
print('Percent of test data = ', len(testlabel)*100/len(dataset.target),'%')

Number of test samples =  4144
Number of other samples =  16496
Percent of test data =  20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [8]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)

What is the accuracy of our classifiers on the train dataset?

In [9]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using nearest neighbour is ", trainAccuracy)

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Train accuracy using random classifier is ", trainAccuracy)

Train accuracy using nearest neighbour is  1.0
Train accuracy using random classifier is  0.164375808538163


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case.

Let us predict the labels for our validation set and get the accuracy

In [10]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour is ", valAccuracy)

valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier is ", valAccuracy)

Validation accuracy using nearest neighbour is  0.34108527131782945
Validation accuracy using random classifier is  0.1688468992248062


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier.

Now let us try another random split and check the validation accuracy

In [11]:
traindata, trainlabel, valdata, vallabel = split(alltraindata, alltrainlabel, 75/100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy of nearest neighbour is ", valAccuracy)

Validation accuracy of nearest neighbour is  0.34048257372654156


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [12]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)
print('Test accuracy is ', testAccuracy)

Test accuracy is  0.34917953667953666


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

1.The accuracy of a model on the validation set can be affected when you increase or reduce the percentage of data allocated to the validation set.

1. **Increase the Percentage of Validation Set:**
   - **Positive Effect:** Increasing the percentage of the validation set typically leads to a more reliable estimate of the model's performance. With more data in the validation set, you get a better sense of how well the model generalizes to unseen data. This can help you detect overfitting and select better hyperparameters, ultimately leading to a more accurate model. However, you'll have less data available for training, so the model may not learn as much, which can be a downside in some cases.

2. **Reduce the Percentage of Validation Set:**
   - **Positive Effect:** Reducing the percentage of the validation set has the opposite effect. More data is available for training, which can lead to a model that learns better from the available data. This can result in a higher accuracy on the training set.
   - **Negative Effect:** However, the downside of reducing the validation set is that it may become less representative of the model's generalization performance. This can make it more challenging to identify overfitting or underfitting, and the estimated accuracy may be less reliable. Additionally, if you reduce the validation set too much, you may not have enough data to validate the model's performance effectively.

The choice of the validation set size is a trade-off between getting an accurate estimate of the model's performance and ensuring that the model has enough data to train effectively. It's often recommended to split the data into training, validation, and test sets in a way that allows for a good balance. A common split is 70-80% for training, 10-15% for validation, and the rest for testing. This provides a reasonable balance between training and validation data to develop a well-performing model while ensuring a good estimate of its performance.

2.The size of the training and validation sets can have an impact on how well you can predict the accuracy on the test set using the validation set. Here's how it can affect the prediction:

1. **Large Training Set and Small Validation Set:**
   - **Effect:** When you have a large training set and a small validation set, the model has a lot of data to learn from, and it may perform well on the training set. However, the small validation set might not be representative enough to accurately estimate the model's generalization performance. This can lead to overfitting, where the model appears to perform well on the validation set but fails to generalize to the test set.
   - **Prediction:** In this case, the validation set may not be a reliable indicator of the model's accuracy on the test set. Predicting accuracy on the test set using the validation set might overestimate the model's true performance.

2. **Small Training Set and Large Validation Set:**
   - **Effect:** With a small training set and a large validation set, the model may not have enough data to learn effectively, which can result in underfitting. The validation set is more representative, but the model's performance on the validation set may not be particularly high.
   - **Prediction:** Using the validation set to predict accuracy on the test set may provide a more accurate estimate, as the validation set is more representative. However, the model's overall performance may still be limited by the small training set.

3. **Balanced Training and Validation Set Sizes:**
   - **Effect:** Ideally, having a balanced training set and validation set with sufficient data in both sets allows the model to learn effectively and provides a reliable estimate of its generalization performance. The model is less likely to overfit or underfit.
   - **Prediction:** When you use a balanced validation set, it becomes a more reliable predictor of the model's performance on the test set. The accuracy on the validation set is a good indicator of how the model will perform on unseen data in the test set.

In practice, it's essential to strike a balance between the sizes of the training and validation sets to ensure a reliable estimate of model performance. This balance can help you make more accurate predictions of accuracy on the test set using the validation set. Cross-validation techniques, like k-fold cross-validation, can also be used to mitigate the impact of the initial split size and provide more robust estimates of model performance.

3.The choice of the percentage to reserve for the validation set depends on several factors, including the size of your overall dataset, the complexity of your model, and the nature of your machine learning problem. However, a common and widely accepted practice is to allocate about 10% to 20% of your data for the validation set. This percentage often strikes a good balance between the two factors you mentioned:

1. **Reliable Model Evaluation:** With 10% to 20% of the data in the validation set, you typically have enough data to reliably evaluate your model's performance and detect issues like overfitting. It provides a reasonable representation of how well your model generalizes to unseen data.

2. **Sufficient Training Data:** You still retain 80% to 90% of your data for training, which provides sufficient data for your model to learn effectively. More data for training is generally better, as it helps your model capture underlying patterns in the data.

Keep in mind that these percentages are not set in stone and may vary depending on your specific situation. Here are some factors to consider when choosing the size of the validation set:

- **Size of the Dataset:** If you have a very large dataset, you can allocate a smaller percentage to the validation set and still have a substantial amount of validation data. Conversely, if your dataset is small, you may need to allocate a larger percentage to ensure a representative validation set.

- **Complexity of the Model:** More complex models, such as deep neural networks, often require larger validation sets to prevent overfitting. In such cases, you might lean towards the higher end of the 10% to 20% range.

- **Data Availability:** In some cases, you may have limited data to work with, and you may need to make trade-offs. If the dataset is small, you might allocate more to the validation set, but be aware that this leaves less data for training.

Ultimately, the choice of the validation set size is often an empirical decision, and you may need to experiment with different percentages to find the one that works best for your specific problem and dataset. Regularly reviewing your model's performance on the validation set and making adjustments as needed is a good practice in model development.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>crossvalidation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute.

In [13]:
# you can use this function for random classifier also
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
  accuracy = 0
  for ii in range(iterations):
    traindata, trainlabel, valdata, vallabel = split(alldata, alllabel, splitpercent)
    valpred = classifier(traindata, trainlabel, valdata)
    accuracy += Accuracy(vallabel, valpred)
  return accuracy/iterations # average of all accuracies

In [14]:
print('Average validation accuracy is ', AverageAccuracy(alltraindata, alltrainlabel, 75/100, 10, classifier=NN))
testpred = NN(alltraindata, alltrainlabel, testdata)
print('test accuracy is ',Accuracy(testlabel, testpred) )

Average validation accuracy is  0.33584635395170215
test accuracy is  0.34917953667953666


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?


1. Cross-validation is a robust technique for model evaluation and selection for several reasons:

1. **Reduced Variability:** Cross-validation helps reduce the variability in model evaluation. By performing multiple train-validation splits and averaging the results, you get a more stable estimate of your model's performance. This is particularly important when you have a limited amount of data, as it allows you to make better use of the available samples.

2. **Better Generalization Assessment:** With cross-validation, your model is evaluated on multiple subsets of the data, which provides a more comprehensive assessment of its ability to generalize to unseen data. This helps you avoid making decisions based on a single random split that may not be representative of the dataset's overall characteristics.

3. **Mitigating Data Biases:** In a single random split, there's a chance that the data in the validation set may be skewed or biased in some way, leading to overly optimistic or pessimistic estimates of model performance. Cross-validation helps mitigate this risk by ensuring that all data points are used for both training and validation over multiple folds.

4. **Optimal Hyperparameter Tuning:** Cross-validation is particularly valuable when tuning hyperparameters. It allows you to evaluate different hyperparameter settings on multiple validation sets, helping you choose the best configuration that is robust across various data subsets.

Common variants of cross-validation include k-fold cross-validation (where the data is divided into k subsets, and the model is trained and evaluated k times), stratified cross-validation (which maintains class distribution in each fold), and leave-one-out cross-validation (LOOCV, where each data point serves as its validation set in turn). The choice of the specific cross-validation technique depends on your data and problem.

2.Cross-validation, specifically k-fold cross-validation, can provide a more accurate estimate of the test accuracy than a single train-validation split when you don't have access to a separate test set. Here's why:

1. **Reduced Variance:** Averaging the results of multiple validation sets generated through k-fold cross-validation reduces the variance in the accuracy estimate. By training and testing the model on different subsets of the data, you get a more stable and less sensitive estimate of the model's performance. This helps to mitigate the impact of random data splits.

2. **More Comprehensive Assessment:** Cross-validation provides a more comprehensive assessment of the model's generalization ability. It evaluates the model on multiple partitions of the data, allowing you to see how well it performs across different subsets. This is particularly important when your data has inherent variations or biases, as cross-validation provides a more representative view of the model's capabilities.

3. **Optimal Hyperparameter Tuning:** Cross-validation is valuable for hyperparameter tuning. It allows you to assess how different hyperparameter settings perform across multiple validation sets, helping you select hyperparameters that are robust and generalize well to unseen data.

However, it's essential to keep in mind that while cross-validation provides a more accurate estimate of how your model is likely to perform on unseen data, it is not a direct estimate of the test accuracy. The test accuracy can only be determined by evaluating the model on a truly independent test dataset that the model has not seen during training or hyperparameter tuning.

If you have access to a separate test set, it is typically best to reserve it for final model evaluation after you have selected and fine-tuned your model using cross-validation. The test accuracy on this holdout test set will give you the most accurate estimate of how your model is expected to perform on completely new, unseen data.

3.The number of iterations or folds in cross-validation can have an impact on the estimate of model performance. In general, increasing the number of iterations (i.e., using more folds) can provide a more robust estimate of model performance, but there are diminishing returns, and it also comes at the cost of increased computation time.

Here are some key points to consider regarding the effect of the number of iterations on the estimate:

1. **Fewer Iterations (e.g., 5-Fold Cross-Validation):**
   - **Advantages:** Fewer iterations require less computation time, making it faster to perform cross-validation. It can be a good choice when you have limited computational resources.
   - **Disadvantages:** The estimate of model performance can be more sensitive to the specific data splits. With fewer iterations, there's a higher chance that the data in one split may not be representative of the overall dataset, leading to less reliable estimates.

2. **More Iterations (e.g., 10-Fold or 20-Fold Cross-Validation):**
   - **Advantages:** More iterations provide a more stable and less variable estimate of model performance. It's less sensitive to the choice of data splits, and the estimate is generally more reliable.
   - **Disadvantages:** Using more iterations requires more computational resources and time. The law of diminishing returns applies, meaning that as you increase the number of iterations, the improvements in estimate stability become less significant.

3. **Leave-One-Out Cross-Validation (LOOCV):**
   - **Advantages:** LOOCV is the extreme case of cross-validation where you leave out one data point at a time, resulting in as many iterations as there are data points. It provides the least biased estimate of model performance but can be computationally expensive.
   - **Disadvantages:** LOOCV can be computationally intensive, especially with large datasets. It may not be practical in some cases.

In practice, the choice of the number of iterations in cross-validation often depends on a trade-off between computational resources, the desire for a more robust estimate, and the nature of your dataset. A common choice is 5-fold or 10-fold cross-validation, as it strikes a good balance between computation time and reliable estimates. However, for small datasets, you might consider using leave-one-out cross-validation for a less biased estimate.

While more iterations generally lead to a more stable estimate, it's important to remember that the primary goal of cross-validation is to assess your model's performance and compare different models or hyperparameters. The exact value of the estimate is less important than its relative comparisons between different models or settings.

4.Increasing the number of iterations in cross-validation can help mitigate the impact of having a very small training dataset or validation dataset to some extent, but it may not fully address the limitations of a very small dataset. Here's how the number of iterations can affect the situation:

1. **Mitigating Variability:** With more iterations, you'll have multiple opportunities to create different training and validation subsets from your small dataset. This can help reduce the variability in the cross-validation results. It may provide more stable and less biased estimates of model performance compared to a single train-validation split.

2. **Representativeness:** Increasing the number of iterations can increase the likelihood of including different subsets of your small dataset in the training and validation sets. This can help ensure that the performance estimates are more representative of the entire dataset.

However, there are some limitations to consider:

1. **Small Training Dataset:** If your training dataset is very small, increasing the number of iterations won't magically generate more training data. Your model may still struggle to learn meaningful patterns from a limited amount of training data, which can lead to underfitting.

2. **Small Validation Dataset:** Similarly, if your validation dataset is extremely small, each validation split may not provide a reliable estimate of your model's generalization performance. While more iterations can help, it may not fully address the limitations of having a very small validation dataset.

In situations where you have a very small dataset, it's essential to be aware of the inherent limitations and consider the following:

- **Data Augmentation:** If applicable, consider data augmentation techniques to artificially increase the size of your training dataset by creating new, slightly modified examples from your existing data.

- **Simpler Models:** With very limited data, it's often wise to opt for simpler models to prevent overfitting. Complex models may struggle to generalize effectively with a small amount of training data.

- **Regularization:** Apply regularization techniques to prevent overfitting and encourage the model to learn more generalizable patterns from the small training dataset.

- **Collect More Data:** If possible, explore options for collecting more data, as increasing the dataset size is one of the most effective ways to improve model performance.

While increasing the number of cross-validation iterations can help make the most of your limited data, it's important to acknowledge the limitations of having a very small dataset and take appropriate steps to address them in your machine learning workflow.